### pivot
- 데이터 프레임에서 두개의 열을 이용하여=> 행과 열 인덱스로 reshpae 된 테이브을 통한 집계
- pivot(index,columns,data.values)
- pivot_table(data,values,index,columns,aggfunc) 추가적으로 margin margin_name 등등 있다

In [1]:
import numpy as np
import pandas as pd

In [7]:
data = {
    'city' : ['seoul','seoul','seoul','pusan','pusan','pusan','gwangju','gwangju'],
    'year' : [2015,2010,2005,2015,2020,2000,2015,2010],
    'pop' : [100,200,300,400,500,600,700,800],
    'region' : ['수도권','수도권','수도권','경상도','경상도','경상도','전라도','전라도']
}
pivot_df = pd.DataFrame(data)
display(pivot_df)

,city,year,pop,region
0,seoul,2015,100,수도권
1,seoul,2010,200,수도권
2,seoul,2005,300,수도권
3,pusan,2015,400,경상도
4,pusan,2020,500,경상도
5,pusan,2000,600,경상도
6,gwangju,2015,700,전라도
7,gwangju,2010,800,전라도


In [8]:
pivot_df.pivot('city','year','pop')

year,2000,2005,2010,2015,2020
city,,,,,
gwangju,NaN,NaN,800.0,700.0,NaN
pusan,600.0,NaN,NaN,400.0,500.0
seoul,NaN,300.0,200.0,100.0,NaN


In [12]:
# 다중 인덱스
pivot_df.pivot(['region','city'],'year','pop')

,year,2000,2005,2010,2015,2020
region,city,,,,,
경상도,pusan,600.0,NaN,NaN,400.0,500.0
수도권,seoul,NaN,300.0,200.0,100.0,NaN
전라도,gwangju,NaN,NaN,800.0,700.0,NaN


#### pivot

In [2]:
# 실습 데이터 로드
import seaborn as sns
iris_df = sns.load_dataset('iris')
titanic_df = sns.load_dataset('titanic')
display(iris_df.head())
display(titanic_df.head())

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [40]:
# 성별과 객실등급에 따른 승객의 수를 집계한다면?
titanic_df[['pclass','sex']].groupby(['sex','pclass']).size()
titanic_df.groupby(['sex','pclass']).size()


sex     pclass
female  1          94
        2          76
        3         144
male    1         122
        2         108
        3         347
dtype: int64

In [137]:
# series를 바로 데이터 프레임으로 만들어보자

# reset_index 활용
# reset_index는 기존의 index가 column으로 튀어나간다
# 그래서 multi index가 아닌 형태로 만들어지게 된다
display(titanic_df.groupby(['sex','pclass']).size().reset_index(name='cnt'))

# 물론 다시 set_index 가능하다
# titanic_df.groupby(['sex','pclass']).size().reset_index(name='cnt').set_index(['sex','pclass'])

display(titanic_df.groupby(['sex','pclass']).size().to_frame(name='cnt'))
#pd.DataFrame(titanic_df.groupby(['sex','pclass']).size())



,sex,pclass,cnt
0,female,1,94
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


cnt
sex    pclass     
female 1        94
       2        76
       3       144
male   1       122
       2       108
       3       347

In [67]:
# dataframe을 다시 pivot table로 만들어보자
titanic_df.groupby(['sex','pclass']).size().reset_index(name='cnt').pivot('sex','pclass')

cnt          
pclass    1    2    3
sex                  
female   94   76  144
male    122  108  347

<hr>

#### pivot_table
- ''로 함수가 들어가는 것들은  'DataFrameGroupBy' object의 원래 함수들이다

In [6]:
# ''로 함수가 들어가는 것들은  'DataFrameGroupBy' object의 원래 함수들이다
# 여기서 'size'는 groupby의 size다

titanic_df.pivot_table(index='sex',columns='pclass',aggfunc='size')


# np.size가 들어가면 안된다
#titanic_df.pivot_table(index=['sex','pclass'],aggfunc=np.size)
#titanic_df.pivot_table(values=['age'],index=['sex','pclass'],aggfunc=np.size)

pclass,1,2,3
sex,,,
female,94,76,144
male,122,108,347


In [129]:
# 아래 두개 결과 동일
titanic_df.pivot_table(index='sex',columns='pclass',aggfunc='max')
titanic_df.pivot_table(index='sex',columns='pclass',aggfunc=np.max)

/Users/iganghui/opt/anaconda3/lib/python3.8/site-packages/pandas/core/apply.py:577: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  return f(*args, **kwargs)
/Users/iganghui/opt/anaconda3/lib/python3.8/site-packages/pandas/core/apply.py:169: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  return getattr(obj, f)()


adult_male                 age             alive           alone  ...  \
pclass          1      2      3     1     2     3     1    2    3     1  ...   
sex                                                                      ...   
female      False  False  False  63.0  57.0  63.0   yes  yes  yes  True  ...   
male         True   True   True  80.0  70.0  74.0   yes  yes  yes  True  ...   

       parch sibsp       survived          who                
pclass     3     1  2  3        1  2  3      1      2      3  
sex                                                           
female     6     3  3  8        1  1  1  woman  woman  woman  
male       5     3  2  8        1  1  1    man    man    man  

[2 rows x 27 columns]

In [130]:
# 실스 데이터
tips_df = sns.load_dataset('tips')
tips_df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [89]:
tips_df['tip']/tips_df['total_bill']

0      1.01
1      1.66
2      3.50
3      3.31
4      3.61
       ... 
239    5.92
240    2.00
241    2.00
242    1.75
243    3.00
Name: tip, Length: 244, dtype: float64

In [82]:
# 식사대금 대비 팁의 비율이 가장 높은 데이터를 찾는다면?
tips_df.loc[pd.Series.idxmax(tips_df['tip']/tips_df['total_bill'])]

total_bill      7.25
tip             5.15
sex             Male
smoker           Yes
day              Sun
time          Dinner
size               2
Name: 172, dtype: object

In [90]:
# 성별에 따른 갯수
tips_df.groupby('sex').size()

sex
Male      157
Female     87
dtype: int64

### cut 
- qcut() : 동일한 갯수의 범주를 만든다
- cut() : 동일한 길이의 범주를 만든다

In [139]:
iris_df = sns.load_dataset('iris')
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [144]:
iris_grp = iris_df.groupby('species')
for key,group in iris_grp:
    print(key)
    print(group)

setosa
    sepal_length  sepal_width  petal_length  petal_width species
0            5.1          3.5           1.4          0.2  setosa
1            4.9          3.0           1.4          0.2  setosa
2            4.7          3.2           1.3          0.2  setosa
3            4.6          3.1           1.5          0.2  setosa
4            5.0          3.6           1.4          0.2  setosa
5            5.4          3.9           1.7          0.4  setosa
6            4.6          3.4           1.4          0.3  setosa
7            5.0          3.4           1.5          0.2  setosa
8            4.4          2.9           1.4          0.2  setosa
9            4.9          3.1           1.5          0.1  setosa
10           5.4          3.7           1.5          0.2  setosa
11           4.8          3.4           1.6          0.2  setosa
12           4.8          3.0           1.4          0.1  setosa
13           4.3          3.0           1.1          0.1  setosa
14           5.8  

In [145]:
iris_df.groupby('species').petal_length.size()

species
setosa        50
versicolor    50
virginica     50
Name: petal_length, dtype: int64

In [149]:
pd.cut(iris_df.petal_length,bins=3,labels=['대','중','소'])

0      대
1      대
2      대
3      대
4      대
      ..
145    소
146    소
147    소
148    소
149    소
Name: petal_length, Length: 150, dtype: category
Categories (3, object): ['대' < '중' < '소']

In [156]:
## 각 분꽃 꽃잎의 길이가 해당 종내에서 대중소 중 어느 그룹에 속하는지 알고싶다면

def cut_petal(s):
    return pd.qcut(s,3,labels=['대','중','소'])

iris_df.groupby('species').petal_length.transform(cut_petal)

0      대
1      대
2      대
3      중
4      대
      ..
145    대
146    대
147    대
148    중
149    대
Name: petal_length, Length: 150, dtype: category
Categories (3, object): ['대' < '중' < '소']

In [163]:
iris_df.groupby('species').petal_length.transform(pd.cut,bins=3,labels=['대','중','소'])

0      중
1      중
2      대
3      중
4      중
      ..
145    대
146    대
147    대
148    중
149    대
Name: petal_length, Length: 150, dtype: category
Categories (3, object): ['대' < '중' < '소']

In [157]:
iris_df.groupby('species').petal_length.apply(cut_petal)

0      대
1      대
2      대
3      중
4      대
      ..
145    대
146    대
147    대
148    중
149    대
Name: petal_length, Length: 150, dtype: category
Categories (3, object): ['대' < '중' < '소']

In [161]:
iris_df.groupby('species').petal_length.apply(pd.cut,bins=3,labels=['대','중','소'])

0      중
1      중
2      대
3      중
4      중
      ..
145    대
146    대
147    대
148    중
149    대
Name: petal_length, Length: 150, dtype: category
Categories (3, object): ['대' < '중' < '소']

In [178]:
# 종별로 평균을 구한다면

iris_df.groupby('species').mean()
iris_df.groupby('species').agg('mean')
iris_df.groupby('species').agg(np.mean)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [186]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [188]:
titanic_df['age_grp']=pd.qcut(titanic_df['age'],3,['A','B','C']) # Nan 값은 제외하고 계산하는거 같다
titanic_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_grp
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,A
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,C
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,B
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,C
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,C


In [194]:
# 성별, 선실, 나이그룹(age_qcut) 의한 생존율을 데이터프레임으로 생성하고
# 행에는 성별 및 나이그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다
titanic_df.pivot_table(values='survived',index =['sex','age_grp'],columns=['pclass'],aggfunc='mean')

pclass                 1         2         3
sex    age_grp                              
female A        0.954545  1.000000  0.508475
       B        0.947368  0.909091  0.481481
       C        0.977273  0.857143  0.250000
male   A        0.500000  0.357143  0.158879
       B        0.500000  0.076923  0.195652
       C        0.347826  0.062500  0.055556

In [195]:
# 성별 및 선실에 의한 생존율을 피벗 데이터 형태로 만든다
titanic_df.pivot_table(values='survived',index =['sex'],columns=['pclass'],aggfunc='mean')

pclass,1,2,3
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447
